In [1]:
import numpy as np
from numpy import array as arr
from tabulate import tabulate

In [2]:
import sys
import socket
if socket.gethostname() == 'Birks-iMac.fritz.box':
    sys.path.append('/Users/Birk/Documents/Programmieren/Python/Eigener Code')
else:
    sys.path.append('/Home/ab227/linux/Eigener Code')
from pap import *   # Eigene Funktionen zur Fehlerberechnung und Resulat-Darstellung
from pap import _rundung, _istbool

# pap.vergleichstabelle Tests

## Status des Projekts
* Skizzen, Planung und Übersicht ist auf dem iPad in 'Notizen Pap II'
* Gerade wird an Punkt 5 - "Rundung hinbekommen" gearbeitet.
* Dazu wird gerade versucht, das nachkommastellen-array in die String-Formatierungen bei "# Darstellung der Werte" einfließen zu lassen.

In [3]:
def vergleichstabelle(werte:np.ndarray, einheit = '', faktor = 1):
    # Vorbearbeitung der Werte
    if np.shape(werte) == (4,):   
        werte = np.array(werte, ndmin = 2).T
    werte = werte * faktor   # Umrechnung auf gewünschte Größenordnung bzw. Einheit
    werte[1::2] = np.abs(werte[1::2])      # Keine negativen Fehlerwerte
    theo, theo_fehler, ex, ex_fehler = werte
    
    
    # Ausrechnen der Abweichungen
    abweichung_abs = ex - theo
    abweichung_abs_fehler = summen_fehler(arr([theo_fehler, ex_fehler]))
    abweichung_rel = abweichung_abs / theo * 100   # [%]
    abweichung_sig = np.abs(abweichung_abs_fehler / abweichung_abs)
    
    '''
    abweichung_rel = abweichung_abs / theo * 100  if theo != 0  else  None   # [%]
    abweichung_sig = np.abs(abweichung_abs_fehler / abweichung_abs)  if abweichung_abs != 0  else  None
    if theo != 0:
        abweichung_rel = abweichung_abs / theo * 100   # [%]
    else:
        abweichung_rel = None
    if abweichung_abs != 0:
        abweichung_sig = np.abs(abweichung_abs_fehler / abweichung_abs)
    else:
        abweichung_rel = None'''
   

    # Rundung der Werte
    nachkommastellen = []
    for fehler_liste in [theo_fehler, ex_fehler, abweichung_abs_fehler]:
        größenordnung_liste = arr((np.floor(np.log10(fehler_liste))), dtype = 'float64')
        signifikante_stellen = np.ones(len(fehler_liste))
        i = 0
        for fehler in fehler_liste:
            if fehler / 10 ** größenordnung_liste[i] < 4.0:
                signifikante_stellen[i] = 2
            i += 1
        nachkommastellen_liste = -größenordnung_liste + signifikante_stellen - 1    
        nachkommastellen.append(nachkommastellen_liste)
    nachkommastellen = np.array(nachkommastellen)
    #print(nachkommastellen)  
    
    
    # Darstellung der Werte
    werte_aufzählung = np.arange(np.shape(werte)[1])
    print('')
    print([f'{theo[i]} +/- {theo_fehler[i]}' for i in werte_aufzählung])
    print([f'{ex[i]} +/- {ex_fehler[i]}' for i in werte_aufzählung])
    print([f'{abweichung_abs[i]} +/- {abweichung_abs_fehler[i]}'  for i in werte_aufzählung])
    print([f'{abweichung_rel[i]} %' for i in werte_aufzählung])
    print([f'{abweichung_sig[i]}' for i in werte_aufzählung])


'''
if nachkommastellen <= 0:
        wertepaar = np.int_(np.round(neue_werte, nachkommastellen))
        nachkommastellen = 0
    else:
        wertepaar = np.round(neue_werte, nachkommastellen) 
    
    print(wertepaar)
    # Print des Ergebnis-Strings
    if len(werte) == 1:
        print(titel + ': {:.{prec}f} {}'.format(*[*wertepaar, einheit], 
              prec = nachkommastellen))
    elif len(werte) == 2:
        print(titel + ': {:.{prec}f} +/- {:.{prec}f} {}'.format(*[*wertepaar, einheit], 
              prec = nachkommastellen))
    elif len(werte) == 3:
        print(titel + ': {:.{prec}f} +/- {:.{prec}f}(sys) +/- {:.{prec}f}(stat) {}'
              .format(*[*wertepaar, einheit], prec = nachkommastellen))
    else:
        print('Zu viele Elemente in werte! >:(')
'''




"\nif nachkommastellen <= 0:\n        wertepaar = np.int_(np.round(neue_werte, nachkommastellen))\n        nachkommastellen = 0\n    else:\n        wertepaar = np.round(neue_werte, nachkommastellen) \n    \n    print(wertepaar)\n    # Print des Ergebnis-Strings\n    if len(werte) == 1:\n        print(titel + ': {:.{prec}f} {}'.format(*[*wertepaar, einheit], \n              prec = nachkommastellen))\n    elif len(werte) == 2:\n        print(titel + ': {:.{prec}f} +/- {:.{prec}f} {}'.format(*[*wertepaar, einheit], \n              prec = nachkommastellen))\n    elif len(werte) == 3:\n        print(titel + ': {:.{prec}f} +/- {:.{prec}f}(sys) +/- {:.{prec}f}(stat) {}'\n              .format(*[*wertepaar, einheit], prec = nachkommastellen))\n    else:\n        print('Zu viele Elemente in werte! >:(')\n"

In [4]:
# Test der Funktion:
vergleichstabelle(arr([0.123, 0.012, 1.098, 0.987]))
vergleichstabelle(arr([-0.123, 0.012, 1.098, 0.987]))
vergleichstabelle(arr([-0.123, 0.012, -1.098, 0.987]))
vergleichstabelle(arr([0.123, -0.012, 1.098, 0.987]))
vergleichstabelle(arr([[0.123, 400 / 7], 
                       [0.012, 23 / 7], 
                       [1.098, 337 / 7], 
                       [0.987, 29 / 7]]))


['0.123 +/- 0.012']
['1.098 +/- 0.987']
['0.9750000000000001 +/- 0.9870729456326923']
['792.6829268292684 %']
['1.0123825083412228']

['-0.123 +/- 0.012']
['1.098 +/- 0.987']
['1.221 +/- 0.9870729456326923']
['-992.6829268292684 %']
['0.8084135508867258']

['-0.123 +/- 0.012']
['-1.098 +/- 0.987']
['-0.9750000000000001 +/- 0.9870729456326923']
['792.6829268292684 %']
['1.0123825083412228']

['0.123 +/- 0.012']
['1.098 +/- 0.987']
['0.9750000000000001 +/- 0.9870729456326923']
['792.6829268292684 %']
['1.0123825083412228']

['0.123 +/- 0.012', '57.142857142857146 +/- 3.2857142857142856']
['1.098 +/- 0.987', '48.142857142857146 +/- 4.142857142857143']
['0.9750000000000001 +/- 0.9870729456326923', '-9.0 +/- 5.287644435234785']
['792.6829268292684 %', '-15.75 %']
['1.0123825083412228', '0.5875160483594206']


In [5]:
np.dtype(np.array([1])[0])

dtype('int64')

# 

# 

In [6]:
def vergleichstabelle(werte_array:np.ndarray, titel:str = 'Vergleichstabelle', 
                       headers:list = [''], wertetitel:list = ['exp. Wert', 'the. Wert'], faktor = 1,
                       float_format:str = '.2f'):
    '''werte_array muss ein  Numpy Array mit shape = (4, X) oder (4,) sein.
    '''
    
    if np.shape(werte_array) == (4,):
        werte_array = np.array(werte_array, ndmin = 2).T
    wert1, wert1_fehler, wert2, wert2_fehler = werte_array * faktor
    
    abweichung     = wert1 - wert2
    abweichung_rel = abweichung / wert2 * 100   # [%], relative Abweichung 
    sigma          = np.abs(abweichung / np.sqrt(wert1_fehler**2 + wert2_fehler**2))

    print(f'{titel}\n')
    print(tabulate([[wertetitel[0], *wert1], ['dessen Fehler', *wert1_fehler], [wertetitel[1], *wert2], 
                    ['dessen Fehler', *wert2_fehler], ['rel. Abweichung [%]', *abweichung_rel], 
                    ['Sigma', *sigma]], headers = headers, floatfmt = float_format))

In [7]:
def listen_vergleichstabelle(titel:str, werte:np.ndarray, 
                             nachkommastellen:int = 0, umrechnungsfaktor = 1, ordnungswerte = []):
    if np.shape(werte)[0] == 3:
        liste1, liste2, liste2_fehler = werte * umrechnungsfaktor
        liste1_fehler = np.zeros(len(liste1))
    elif np.shape(werte)[0] == 4:
        liste1, liste1_fehler, liste2, liste2_fehler = werte * umrechnungsfaktor
    else:
        print('Format noch nicht definiert oder werte falsch eingegeben.')
        
    abweichung     = liste2 - liste1
    abweichung_rel = abweichung / liste1 * 100   # [%], relative Abweichung 
    sigma          = np.abs(abweichung / np.sqrt(liste1_fehler**2 + liste2_fehler**2))

    
    if np.shape(werte)[0] == 3:
        liste1_strings = ['{wert:.{prec}f}'.format(wert = liste1[i], prec = nachkommastellen)
                          for i in range(len(liste1))]
    elif np.shape(werte)[0] == 4:
        liste1_strings = ['{wert:.{prec}f} +/- {fehler:.{prec}f}'.format(wert = liste1[i], 
                                                             fehler = liste1_fehler[i], 
                                                             prec = nachkommastellen)
                          for i in range(len(liste1))]
    
    liste2_strings = ['{wert:.{prec}f} +/- {fehler:.{prec}f}'.format(wert = liste2[i], 
                                                             fehler = liste2_fehler[i], 
                                                             prec = nachkommastellen)
                      for i in range(len(liste2))]
        
        
    print(f'{titel}\n')
    if ordnungswerte == []:
        print(tabulate(arr([liste1_strings, liste2_strings, abweichung_rel, sigma]).T, 
                       headers = ('theoretisch', 'experimentell', 'Abweichung [%]', 'Sigma'), 
                       floatfmt = '.1f', stralign = 'right', disable_numparse = [0]))
    else:
        print(tabulate(arr([ordnungswerte[1:], liste1_strings, liste2_strings, abweichung_rel, sigma]).T, 
                       headers = (ordnungswerte[0], 'theoretisch', 'experimentell', 'Abweichung [%]', 'Sigma'), 
                       floatfmt = '.1f', stralign = 'right', disable_numparse = [1]))